# Fix Classes=Categories
version: 3

info: 
- fix classes:
    - find and fix duplicates:
        - equal names
        - similar names

author: nuno costa

In [1]:
from annotate_v5 import *
import platform 
import numpy as np
import pandas as pd
from IPython.display import Image, display
import copy
import os
from shutil import copyfile
import matplotlib.pyplot as plt
from matplotlib.image import imread
from matplotlib.patches import Rectangle
import random

#Define root dir dependent on OS
rdir='D:/external_datasets/' 
if str(platform.platform()).find('linux')>-1: rdir='/mnt/d/external_datasets/' 
print('OS: {}'.format(platform.platform()))
print('root dir: {}'.format(rdir))

OS: Windows-10-10.0.20241-SP0
root dir: D:/external_datasets/


In [40]:
molajson =  json.load(open(rdir+'mola.json'))

In [41]:
for k in molajson:
    print(k, len(molajson[k]))

info 5
licenses 9
categories 1310
videos 1488
images 177936
tracks 8132
segment_info 0
annotations 1338002
datasets 2


## 1. Import ids
#### #NOTE: work with ids and index so you can use numpy for faster operations
#### #WARNING don't use try: except: pass when importing - go back to mergedatasets and find the BUG. 

In [9]:
# datasets name and id
dset_l=[]
dset_l_id=[]
for d in molajson['datasets']:
    dset_l.append(d['name'])
    dset_l_id.append(d['id'])
print(dset_l, dset_l_id)

['COCO', 'TAO'] [1, 2]


In [10]:
# categories name and id
cat_l=[]
cat_l_id=[]
cat_l_dset=[]
for c in molajson['categories']:
    cat_l.append(c['name'])
    cat_l_id.append(c['id'])
    cat_l_dset.append(dset_l[c['dataset']-1]) # dset_l index is same as id-1
#print(cat_l_id)

In [11]:
# images filepath and id
img_l=[]
img_l_id=[]
for c in molajson['images']:
    img_l.append(c['file_name'])
    img_l_id.append(c['id'])

In [12]:
# annotations category_id, image_id, bbox, and dataset
ann_catid=[]
ann_imgid=[]
ann_bbox=[]
ann_dset=[]
for an in tqdm(molajson['annotations']):
    ann_catid.append(an['category_id'])
    ann_imgid.append(an['image_id'])
    ann_bbox.append(an['bbox'])
    ann_dset.append(an['dataset'])

100%|█████████████████████████████████████████████████████████████████| 1338002/1338002 [00:02<00:00, 454771.31it/s]


## 2. Find duplicates cat_ids
Duplicates example
categories= [{name:cow, id:1, dataset:1},...,{name:cow, id:200, dataset:2},...,{name:cow, id:101, dataset:3}]}

In [13]:
#duplicates #TODO: SORT alphabetically
duplicates_l=[]
duplicates_l_catid=[]
duplicates_l_catdset=[]
duplicate_method="equal_names"
if duplicate_method=="equal_names": #FIX REPETITIONS
    duplicates_l=list(set([x for x in cat_l if cat_l.count(x) > 1])) # duplicates l 
    for duplicate in tqdm(duplicates_l):
        idx_mask = [name == duplicate for name in cat_l] #mask of index of duplicate
        catids = np.array(cat_l_id)[idx_mask] #duplicate catids
        catdsets = np.array(cat_l_dset)[idx_mask] #duplicate catdset
        duplicates_l_catid.append(catids.tolist())
        duplicates_l_catdset.append(catdsets.tolist())
if duplicate_method=="similar_names": #FIX SIMILAR NAMES
    #WARNING: fix equal_names first - if not, it will not be fixed in this case
    import difflib
    for cat in cat_l: 
        match_l=[]
        match_l_temp=[c for c in cat_l if (c).find(cat)>-1] #only substring inside string - catagory with big name is not found
        if not len(match_l_temp)>1: continue #more than one
        #refine search #ADD or REMOVE refine options
        match_l.append(cat) #make category to be the first item
        match_l.extend([c for c in match_l_temp if ((c+' ').find(cat)>-1 or (c+' ').find(cat)>-1 or (' '+c).find(cat)>-1 or (c+'_').find(cat)>-1 or ('_'+c).find(cat)>-1 )]) #mantain optins: c+'_';'_'+c 
        match_l.extend(difflib.get_close_matches(cat, match_l_temp, n=5, cutoff=0.9)) #get similar words
        #remove equal names
        match_l=list(dict.fromkeys(match_l))#match_l=list(set(match_l))
        if not len(match_l)>1: continue #more than one
        #add to duplicates
        duplicates_l.append(match_l)
    for duplicates in tqdm(duplicates_l):
        catids=[]
        catdsets=[]
        for duplicate in duplicates:
            idx=cat_l.index(duplicate) #first duplicate ()
            catids.append(cat_l_id[idx]) #duplicate catids
            catdsets.append(cat_l_dset[idx]) #duplicate cat dsets
        duplicates_l_catid.append(catids)
        duplicates_l_catdset.append(catdsets)
if duplicate_method=="all_names": #Do for all category names, even with equal 
    duplicates_l=cat_l #NORMALIZE: same list format ?? not needed
    duplicates_l_catid=[[id] for id in cat_l_id]
    duplicates_l_catdset=[[dset] for dset in cat_l_dset]
    

print(duplicates_l)
print(duplicates_l_catid)
print(duplicates_l_catdset)
print(len(duplicates_l))
print(len(duplicates_l_catid))
print(len(duplicates_l_catdset))

100%|█████████████████████████████████████████████████████████████████████████████| 49/49 [00:00<00:00, 1689.82it/s]

['boat', 'sheep', 'frisbee', 'carrot', 'cup', 'vase', 'toilet', 'scissors', 'fork', 'toothbrush', 'apple', 'knife', 'dog', 'bed', 'sink', 'skateboard', 'motorcycle', 'broccoli', 'giraffe', 'bowl', 'truck', 'bench', 'bird', 'kite', 'suitcase', 'sandwich', 'backpack', 'spoon', 'chair', 'bear', 'cat', 'umbrella', 'cow', 'cake', 'refrigerator', 'bicycle', 'horse', 'book', 'toaster', 'pizza', 'bottle', 'elephant', 'banana', 'airplane', 'surfboard', 'clock', 'handbag', 'zebra', 'snowboard']
[[9, 198], [19, 1041], [30, 560], [52, 301], [42, 427], [76, 1242], [62, 1197], [77, 1017], [43, 555], [80, 1202], [48, 93], [44, 705], [17, 462], [60, 159], [72, 1059], [37, 1060], [4, 794], [51, 234], [24, 582], [46, 219], [8, 1224], [14, 171], [15, 179], [34, 701], [29, 116], [49, 1006], [25, 114], [45, 1098], [57, 317], [22, 158], [16, 309], [26, 1235], [20, 161], [56, 265], [73, 508], [2, 175], [18, 659], [74, 206], [71, 1195], [54, 909], [40, 213], [21, 509], [47, 125], [5, 84], [38, 1137], [75, 356

In [14]:
# get annotations duplicates
ann_catid_np=np.array(ann_catid)
ann_imgid_np=np.array(ann_imgid)
ann_bbox_np=np.array(ann_bbox)
ann_dset_np=np.array(ann_dset)
duplicates_l_imgid=[]
duplicates_l_bbox=[]
duplicates_l_dset=[]
for catids in tqdm(duplicates_l_catid):
    l_imgid=[]
    l_bbox=[]
    l_dset=[]
    for catid in catids:
        ann_idx = np.where(ann_catid_np==catid)[0].tolist() #annotation index of ids
        l_imgid.append(ann_imgid_np[ann_idx].tolist())
        l_bbox.append(ann_bbox_np[ann_idx].tolist())
        l_dset.append(ann_dset_np[ann_idx].tolist())
    duplicates_l_imgid.append(l_imgid)
    duplicates_l_bbox.append(l_bbox)
    duplicates_l_dset.append(l_dset)

100%|███████████████████████████████████████████████████████████████████████████████| 49/49 [00:00<00:00, 84.93it/s]


## 3. Classes|categories to fix 

In [15]:
classtofix_l=[]
classtofix_l_catid=[]
method="all"
if method=="all": #fix all duplicates
    classtofix_l=duplicates_l
    classtofix_l_catid=duplicates_l_catid
if method=="view_images": # view images and select class to fix (INCEPTION =>>>Pufff!!!!)
    img_l_id_np=np.array(img_l_id)      
    for i, duplicate in enumerate(tqdm(duplicates_l)): #run for each duplicate
        print(duplicate+'...') #class
        display_imgs(rdir, i, dset_l, duplicates_l_catid, duplicates_l_dset, duplicates_l_imgid, img_l, img_l_id, imgidx=0)
        classtofix_l, classtofix_l_catid=save_classtofix(i, classtofix_l, classtofix_l_catid, duplicates_l, duplicates_l_catid, duplicates_l_dset, duplicates_l_imgid, img_l, img_l_id, imgidx=0)
if method=="save_images": # save images to folder for manual check
    datadir="duplicates"
    folder=duplicate_method+'/'
    showimage=False #show images
    startidx=0 # start index of image to save from each dataset
    imgnr=10 # total number of images to save from each dataset
    imgstep='random' # step between images: int | 'random' - int steps between images; 'rand' gets random list
    path=os.path.join(rdir,datadir,folder)
    assure_path_exists(path)
    #TODO
    #save excel # TODO Send duplicates_l, duplicates_L_catid, duplicates_l_catdset: user should make column  classtofix_l and classtofix_L_catid - user should create  
    df=pd.DataFrame({'duplicates_l': duplicates_l,'duplicates_l_catid': duplicates_l_catid, 'duplicates_l_catdset': duplicates_l_catdset, 'classtofix_l': np.nan, 'classtofix_l_catid':np.nan, 'rules':np.nan })
    df.loc[0, 'rules']="To fix classes: 1) You need to fill the column classtofix_l and/or classtofix_l_catid with the information from the respective duplicate columns; 2) When copy/pasting or changing, make sure the same structure maintains:  ['car', 'carrot'], [3, 52], beware of the spaces ['car', '  and always maintain the first class in the list;  3) You have 3 possibilities of filling the columns : 1-the 2 columns empty, meaning the row will not be used for classtofix; 2-only one column empty, e.g. fill the classtotix_l row with the class labels from duplicates_l, then during the importing the classtofix_l_catid is filled, and vice-versa; 3-If you want to change the name of the first class in the list,e.g ['car', 'carrot'] for ['automobile', 'carrot'] you need to provide the ids to classtofix_l_catid."
    excelpath=path+duplicate_method+"_classtofix_report.xlsx"
    df['annotations_missing'] = np.empty((len(df), 0)).tolist()
    df['images_missing'] = np.empty((len(df), 0)).tolist()
    #save image for each duplicate
    for i, duplicate in enumerate(tqdm(duplicates_l)): #run for each duplicate category
        minclass=duplicate
        if isinstance(minclass, list): minclass=minclass[0]
        print('\n>> '+minclass+'...') #class
        classpath=os.path.join(path, minclass) # folder for images of this class
        classpath=parse_path(classpath)+'/' #make it a folder
        assure_path_exists(classpath)
        df=save_imgs(df, rdir, classpath, i, dset_l, duplicates_l, duplicates_l_catid, duplicates_l_bbox, duplicates_l_dset,
              duplicates_l_imgid, img_l, img_l_id, startidx=startidx, imgnr=imgnr, imgstep=imgstep, showimage=showimage)
    df.to_excel(excelpath, index=False)   
if method=="view_duplicate":
    i=-1 #duplicate_l index : last category
    ii=1 #dset_l index : TAO
    step=10 #step images
    dpi=80
    dataset=dset_l[ii]
    category=duplicates_l[i]
    view_duplicate(rdir, i, ii, dataset, category, duplicates_l_imgid, duplicates_l_bbox, img_l, img_l_id, step=step,
                   dpi=dpi)

In [16]:
#WARNING: CHECK EXCEL FIRST
if method=="save_images": # reading classtofix from excel
    classtofix_l=[]
    classtofix_l_catid=[]
    df=pd.read_excel(excelpath)
    classtofix_df=df.loc[:,'classtofix_l']
    classtofix_df_catid=df.loc[:,'classtofix_l_catid']
    display(df)
    # PARSE COLUMNS TO FIX
    classtofix_l=classtofix_df.tolist()
    classtofix_l_catid=classtofix_df_catid.tolist()
    print(classtofix_l_catid)
    #convert strings to lists
    for icl, cl in enumerate(classtofix_l): 
        if isinstance(classtofix_l[icl], str): classtofix_l[icl]=convert_unicode(classtofix_l[icl], method='liststr')
        if isinstance(classtofix_l_catid[icl], str): classtofix_l_catid[icl]=convert_unicode(classtofix_l_catid[icl], method='listnum')
    print(classtofix_l_catid)
    #parse the columns based on the rules    
    for ic, classes in enumerate(classtofix_df):
        #1. Two columns empty - do nothing, maintain
        if  pd.isnull(classtofix_df.iloc[ic]) and pd.isnull(classtofix_df_catid.iloc[ic]): continue
        #2. if only classtofix_l_catid empty - get 
        if not pd.isnull(classtofix_df.iloc[ic]) and pd.isnull(classtofix_df_catid.iloc[ic]):
            classes=convert_unicode(classes, method='liststr')
            cids=[]
            for c in classes:
                cidx=duplicates_l[ic].index(c)
                cid=duplicates_l_catid[ic][cidx]
                cids.append(cid)
            classtofix_l_catid[ic]=cids
            print(classtofix_l_catid)
        #2. if only classtofix_l empty - get
        if pd.isnull(classtofix_df.iloc[ic]) and not pd.isnull(classtofix_df_catid.iloc[ic]): 
            classes=[]
            cids=classtofix_l_catid[ic]
            for c in cids:
                cidx=duplicates_l_catid[ic].index(c)
                clas=duplicates_l[ic][cidx]
                classes.append(clas)
            classtofix_l[ic]=classes
        #3. if both columns not empty - do nothing
        if not pd.isnull(classtofix_df.iloc[ic]) and not pd.isnull(classtofix_df_catid.iloc[ic]): continue

In [17]:
print('>> Make sure everything is correct: 1.Drop NaN if exist, but make sure the index is the same for the two! \n')
fixempty=True
if fixempty:
    classtofix_l=[x for x in classtofix_l if str(x) != 'nan' and str(x) !='[]']
    classtofix_l_catid=[x for x in classtofix_l_catid if str(x) != 'nan' and str(x) !='[]']
print(classtofix_l)
print(classtofix_l_catid)

>> Make sure everything is correct: 1.Drop NaN if exist, but make sure the index is the same for the two! 

['boat', 'sheep', 'frisbee', 'carrot', 'cup', 'vase', 'toilet', 'scissors', 'fork', 'toothbrush', 'apple', 'knife', 'dog', 'bed', 'sink', 'skateboard', 'motorcycle', 'broccoli', 'giraffe', 'bowl', 'truck', 'bench', 'bird', 'kite', 'suitcase', 'sandwich', 'backpack', 'spoon', 'chair', 'bear', 'cat', 'umbrella', 'cow', 'cake', 'refrigerator', 'bicycle', 'horse', 'book', 'toaster', 'pizza', 'bottle', 'elephant', 'banana', 'airplane', 'surfboard', 'clock', 'handbag', 'zebra', 'snowboard']
[[9, 198], [19, 1041], [30, 560], [52, 301], [42, 427], [76, 1242], [62, 1197], [77, 1017], [43, 555], [80, 1202], [48, 93], [44, 705], [17, 462], [60, 159], [72, 1059], [37, 1060], [4, 794], [51, 234], [24, 582], [46, 219], [8, 1224], [14, 171], [15, 179], [34, 701], [29, 116], [49, 1006], [25, 114], [45, 1098], [57, 317], [22, 158], [16, 309], [26, 1235], [20, 161], [56, 265], [73, 508], [2, 175],

## 4. Fix classes

In [18]:
# slow # newjson=copy.deepcopy(molajson) #do deepcopy to compare
# fast
newjson={'categories':[],'annotations':[] }
newjson['categories']=copy.copy(molajson['categories'])
newjson['annotations']=copy.copy(molajson['annotations'])

In [19]:
classtofix_l_catidx=[[cat_l_id.index(id) for id in id_l] for id_l in classtofix_l_catid]
#print(classtofix_l_catidx) # they should be less one, becacuse it is ordered
print(len(classtofix_l_catidx))

49


#### Change molajson['categories']: [{name: , id: }]  

- Use min cat id and remove the other categories (!!!Without ordering again the category id!!!)

- #WARNING  the fixed category will only have the key {dataset: } with the first corresponding dataset. 
   - #SOLUTION change {dataset: int } to {dataset: list } in init_json, than in all the functions that deal with the int make the appropriate changes. Then, you can change molajson['categories']: [dataset: }]  using duplicates_l_catdset

In [21]:
removeidx_l=[]
minidx=0 # get first cat: minimum in equal names and the category used to search in similar_names
for i,id_l in enumerate(tqdm(classtofix_l_catid)): #for each classtofix
    minid=id_l[minidx] # #category id 
    catidx=classtofix_l_catidx[i][minidx]# get cat index of min catid
    if isinstance(classtofix_l[i], list): newjson['categories'][firstcatidx]['name']=classtomix_l[i][minidx] #change name of first id 
    else: newjson['categories'][catidx]['name']=classtofix_l[i] #change name of min id (if changed)
    assert newjson['categories'][catidx]['id']==id_l[minidx] #assert id - it should be the same
    otheridx_l=copy.copy(classtofix_l_catidx[i]) #the idx to remove
    otheridx_l.remove(catidx)
    removeidx_l.extend(otheridx_l) #remove index
removeidx_l=list(dict.fromkeys(removeidx_l)) # no duplicates

100%|███████████████████████████████████████████████████████████████████████████████████████| 49/49 [00:00<?, ?it/s]


In [22]:
#REMOVE - Newjson will be changed
removeitem_l=[newjson['categories'][removeidx] for removeidx in removeidx_l] #items to remove
for removeitem in removeitem_l: newjson['categories'].remove(removeitem) 

In [23]:
#TEST
print(molajson['categories'][13])
print(newjson['categories'][13])
print(molajson['categories'][170])
print(newjson['categories'][170])

{'supercategory': 'outdoor', 'id': 14, 'name': 'bench', 'dataset': 1}
{'supercategory': 'outdoor', 'id': 14, 'name': 'bench', 'dataset': 1}
{'frequency': 'f', 'id': 171, 'synset': 'bench.n.01', 'image_count': 93, 'instance_count': 160, 'synonyms': ['bench'], 'def': 'a long seat for more than one person', 'name': 'bench', 'dataset': 2}
{'frequency': 'c', 'id': 183, 'synset': 'birdhouse.n.01', 'image_count': 0, 'instance_count': 0, 'synonyms': ['birdhouse'], 'def': 'a shelter for birds', 'name': 'birdhouse', 'dataset': 2}


#### change molajson['annotations']: [{category_id: , }] 

In [26]:
ann_catid_np=np.array(ann_catid)
classtofix_l_ann_catidx=[[np.where(ann_catid_np==id)[0].tolist()  for id in id_l] for id_l in classtofix_l_catid]
print(len(classtofix_l_ann_catidx))

49


In [27]:
for i,id_l in enumerate(classtofix_l_catid): #for each classtofix
    minidx=0 # get first catid min
    minid=id_l[minidx]
    ann_catidx= classtofix_l_ann_catidx[i][minidx]# get annotation cat index of min catid
    ann_otheridx_l=copy.copy(classtofix_l_ann_catidx[i])
    ann_otheridx_l.remove(ann_catidx) #the idx to change
    for ann_otheridx in ann_otheridx_l[0]: newjson['annotations'][ann_otheridx]['category_id']=minid
        

In [30]:
#TEST
print(molajson['annotations'][8875])
print(newjson['annotations'][8875])
print(molajson['annotations'][1174300])
print(newjson['annotations'][1174300])

print('\n>> molajson \n')
for k in molajson:
    print(k, len(molajson[k]))

print('\n>> newjson \n')    
for k in newjson:
    print(k, len(newjson[k]))

{'segmentation': [[20.95, 105.11, 20.95, 87.58, 8.43, 80.69, 8.43, 75.06, 38.48, 74.43, 40.35, 66.92, 32.22, 63.79, 39.1, 35.62, 135.51, 37.5, 130.5, 65.04, 128.0, 70.05, 124.87, 108.86, 119.86, 102.6, 119.23, 86.95, 111.72, 86.95, 106.08, 107.61, 104.83, 107.61, 98.57, 90.71, 77.91, 86.33, 64.77, 86.33, 55.38, 90.08, 45.99, 100.73, 45.36, 109.49, 41.61, 110.12, 40.35, 97.6, 35.97, 89.46, 29.71, 86.95, 24.7, 93.84, 24.7, 105.73]], 'area': 5697.957150000001, 'iscrowd': 0, 'image_id': 26929, 'bbox': [8.43, 35.62, 127.08, 74.5], 'category_id': 14, 'id': 8876, 'dataset': 1}
{'segmentation': [[20.95, 105.11, 20.95, 87.58, 8.43, 80.69, 8.43, 75.06, 38.48, 74.43, 40.35, 66.92, 32.22, 63.79, 39.1, 35.62, 135.51, 37.5, 130.5, 65.04, 128.0, 70.05, 124.87, 108.86, 119.86, 102.6, 119.23, 86.95, 111.72, 86.95, 106.08, 107.61, 104.83, 107.61, 98.57, 90.71, 77.91, 86.33, 64.77, 86.33, 55.38, 90.08, 45.99, 100.73, 45.36, 109.49, 41.61, 110.12, 40.35, 97.6, 35.97, 89.46, 29.71, 86.95, 24.7, 93.84, 24.7

### 4. Save fixed json

In [42]:
# fast
molajson['categories']=copy.copy(newjson['categories'])
molajson['annotations']=copy.copy(newjson['annotations'])

In [ ]:
# save
print('\n >> SAVING...')
jsonfile=rdir+'mola_fix_equal.json'
with open(jsonfile, 'w') as f:
    json.dump(molajson, f)
print("JSON SAVED : {} \n".format(jsonfile))